# PyCity Schools Analysis 

##### We defined our source data frames

In [1]:
import pandas as pd
from pathlib import Path
from functools import reduce

In [2]:
# File to load
path_school_data = Path("Resources/schools_complete.csv")
path_student_data = Path("Resources/students_complete.csv")

# Read data files and store into Pandas DataFrame 
school_data = pd.read_csv(path_school_data)
student_data = pd.read_csv(path_student_data)

#combine the data into a single dataset.
complete_school_data = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])


##### We formatted the data so that it could be used in a summary of the district

In [3]:
school_count = len(complete_school_data["school_name"].unique())
student_count = len(complete_school_data["student_name"])
total_budget = complete_school_data["budget"].unique().sum()
avg_math_score= complete_school_data["math_score"].mean()
avg_reading_score = complete_school_data["reading_score"].mean()

In [4]:
passing_math_count = complete_school_data.loc[complete_school_data["math_score"] >= 70].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100

passing_reading_count = complete_school_data.loc[complete_school_data["reading_score"] >= 70].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100

passing_math_reading_count = complete_school_data.loc[(complete_school_data["math_score"] >= 70) & 
                                                      (complete_school_data["reading_score"] >= 70)].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100

In [5]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
summary_data = [school_count, student_count, total_budget, avg_math_score, avg_reading_score, 
                passing_math_percentage, passing_reading_percentage, overall_passing_rate]

summary_columns = ["Total Schools", "Total Students", "Total Budget","Average math score", 
                   "Average reading score", "% passing math" ,"% passing reading", "% Overall Passing"]

district_summary = pd.DataFrame([dict(zip(summary_columns, summary_data))])
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

In [6]:
district_summary['Average math score'] = (district_summary["Average math score"]).map("{:.2f}".format)
district_summary['Average reading score'] = (district_summary["Average reading score"]).map("{:.2f}".format)
district_summary['% passing math'] = (district_summary["% passing math"]).map("{:.2f}%".format)
district_summary['% passing reading'] = (district_summary["% passing reading"]).map("{:.2f}%".format)
district_summary['% Overall Passing'] = (district_summary["% Overall Passing"]).map("{:.2f}%".format)

## District Summary
We began by creating a district summary that provides an overview of all schools and students. It’s important to remember these numerical figures so that you can compare them with the next set of data and draw conclusions.


In [7]:
# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average math score,Average reading score,% passing math,% passing reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.99,81.88,74.98%,85.81%,65.17%


##### We formatted the data so that it could be used in a summary of the Schools

In [8]:
#making the school groupby
school_groupby = complete_school_data.groupby("school_name").agg({
  'type': 'unique', 'student_name': 'count',  'budget': 'unique', 'math_score': 'mean', 'reading_score': 'mean'
})

In [9]:
#We prepared the data so that we could perform mathematical operations on it
school_groupby["type"] = school_groupby["type"].apply(lambda x: ','.join(x))
school_groupby["budget"] = school_groupby["budget"].apply(lambda x:int(x[0]))
school_groupby["Per student budget"] = school_groupby["budget"]/school_groupby["student_name"]

In [10]:
#We analyzed the data and created new columns of information based on our findings
math_pass_df = complete_school_data.loc[(complete_school_data["math_score"] >= 70)]
math_pass_groupby = math_pass_df.groupby("school_name").agg({'math_score': 'count'})
reading_pass_df = complete_school_data.loc[(complete_school_data["reading_score"] >= 70)]
reading_pass_groupby = reading_pass_df.groupby("school_name").agg({'reading_score': 'count'})

school_groupby['% passing math'] = math_pass_groupby['math_score']/school_groupby["student_name"]
school_groupby['% passing reading'] = reading_pass_groupby['reading_score']/school_groupby["student_name"]

In [11]:
#WWe analyzed the data and created new columns of information based on our findings
overall_pass_df = complete_school_data.loc[(complete_school_data["math_score"] >= 70) & (complete_school_data["reading_score"] >= 70)]
overall_pass_groupby = overall_pass_df.groupby('school_name').agg({'reading_score': 'count'})
school_groupby['% Overall Passing'] = overall_pass_groupby['reading_score']/school_groupby["student_name"]

In [12]:
#We duplicated the data so that we could use it in future research
school_spending_df = school_groupby.copy()
school_size_df = school_groupby.copy()
school_type_df = school_groupby.copy()
per_school_summary = school_groupby.copy()

In [13]:
#We formatted the data so that it could be displayed in a clear and concise manner
per_school_summary['math_score'] = (per_school_summary['math_score']).map("{:.2f}".format)
per_school_summary['reading_score'] = (per_school_summary['reading_score']).map("{:.2f}".format)
per_school_summary['% passing math'] = (per_school_summary['% passing math']*100).map("{:.1f}%".format)
per_school_summary['% passing reading'] = (per_school_summary['% passing reading']*100).map("{:.1f}%".format)
per_school_summary['% Overall Passing'] = (per_school_summary["% Overall Passing"]*100).map("{:.1f}%".format)

In [14]:
spending_bins = [0, 585, 630, 645, 680]
labels = ["$000-$585", "$585-630", "$630-645", "$645-680"]
per_school_summary["Spending Ranges (Per Student)"] = pd.cut(per_school_summary['Per student budget'], bins=spending_bins, labels=labels, include_lowest=True)

In [15]:
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
per_school_summary["school size"] = pd.cut(per_school_summary['student_name'], bins=size_bins, labels=labels, include_lowest=True)
per_school_summary.rename(columns={'math_score' : 'Average math score', "reading_score" : "Average reading score"}, inplace=True)

In [16]:
#We formatted the Columns so that it could be displayed in a clear and concise manner
per_school_summary.rename(columns={'type' : 'School type','student_name' : 'Total students', "budget" : "Total school budget",
                               'math_score' : 'Average math score', "reading_score" : "Average reading score"
                               }, inplace=True)


## Per School Summary
This data frame contains detailed information about each school.


In [17]:
# Display the per_school_summary DataFrame
per_school_summary

,School type,Total students,Total school budget,Average math score,Average reading score,Per student budget,% passing math,% passing reading,% Overall Passing,Spending Ranges (Per Student),school size
school_name,,,,,,,,,,,
Bailey High School,District,4976,3124928,77.05,81.03,628.0,66.7%,81.9%,54.6%,$585-630,Large (2000-5000)
Cabrera High School,Charter,1858,1081356,83.06,83.98,582.0,94.1%,97.0%,91.3%,$000-$585,Medium (1000-2000)
Figueroa High School,District,2949,1884411,76.71,81.16,639.0,66.0%,80.7%,53.2%,$630-645,Large (2000-5000)
Ford High School,District,2739,1763916,77.10,80.75,644.0,68.3%,79.3%,54.3%,$630-645,Large (2000-5000)
Griffin High School,Charter,1468,917500,83.35,83.82,625.0,93.4%,97.1%,90.6%,$585-630,Medium (1000-2000)
Hernandez High School,District,4635,3022020,77.29,80.93,652.0,66.8%,80.9%,53.5%,$645-680,Large (2000-5000)
Holden High School,Charter,427,248087,83.80,83.81,581.0,92.5%,96.3%,89.2%,$000-$585,Small (<1000)
Huang High School,District,2917,1910635,76.63,81.18,655.0,65.7%,81.3%,53.5%,$645-680,Large (2000-5000)
Johnson High School,District,4761,3094650,77.07,80.97,650.0,66.1%,81.2%,53.5%,$645-680,Large (2000-5000)


##### We formatted the data so that it could be used in summaries of the highest- and lowest-performing school

In [18]:
# sorting values for bottom schools
per_school_summary = per_school_summary.sort_values(by= '% Overall Passing')
bottom_schools = per_school_summary.head()


In [19]:
# sorting values for top schools
per_school_summary = per_school_summary.sort_values(by= '% Overall Passing', ascending=False)
top_schools = per_school_summary.head()

## Bottom Schools
It’s important to note that five schools of the district school type are considered bottom schools.

In [20]:
# Display bottom schools
bottom_schools

,School type,Total students,Total school budget,Average math score,Average reading score,Per student budget,% passing math,% passing reading,% Overall Passing,Spending Ranges (Per Student),school size
school_name,,,,,,,,,,,
Rodriguez High School,District,3999,2547363,76.84,80.74,637.0,66.4%,80.2%,53.0%,$630-645,Large (2000-5000)
Figueroa High School,District,2949,1884411,76.71,81.16,639.0,66.0%,80.7%,53.2%,$630-645,Large (2000-5000)
Hernandez High School,District,4635,3022020,77.29,80.93,652.0,66.8%,80.9%,53.5%,$645-680,Large (2000-5000)
Huang High School,District,2917,1910635,76.63,81.18,655.0,65.7%,81.3%,53.5%,$645-680,Large (2000-5000)
Johnson High School,District,4761,3094650,77.07,80.97,650.0,66.1%,81.2%,53.5%,$645-680,Large (2000-5000)


## Top Schools
It’s important to note that five schools of the charter school type are considered top schools.

In [21]:
# Display top schools 
top_schools

,School type,Total students,Total school budget,Average math score,Average reading score,Per student budget,% passing math,% passing reading,% Overall Passing,Spending Ranges (Per Student),school size
school_name,,,,,,,,,,,
Cabrera High School,Charter,1858,1081356,83.06,83.98,582.0,94.1%,97.0%,91.3%,$000-$585,Medium (1000-2000)
Thomas High School,Charter,1635,1043130,83.42,83.85,638.0,93.3%,97.3%,90.9%,$630-645,Medium (1000-2000)
Griffin High School,Charter,1468,917500,83.35,83.82,625.0,93.4%,97.1%,90.6%,$585-630,Medium (1000-2000)
Wilson High School,Charter,2283,1319574,83.27,83.99,578.0,93.9%,96.5%,90.6%,$000-$585,Large (2000-5000)
Pena High School,Charter,962,585858,83.84,84.04,609.0,94.6%,95.9%,90.5%,$585-630,Small (<1000)


##### We formatted the data so that it could be used in summaries of the average math and reading scores for each grade level

In [22]:
# group by grade and school name for average math score table 
avg_math_score = complete_school_data.groupby(['school_name', 'grade'])['math_score'].mean().reset_index()
pivot1 = avg_math_score.pivot_table(index='school_name', columns='grade', values='math_score')
# formmating data to display
pivot1['9th'] = (pivot1["9th"]).map("{:.2f}".format)
pivot1['10th'] = (pivot1["10th"]).map("{:.2f}".format)
pivot1['11th'] = (pivot1["11th"]).map("{:.2f}".format)
pivot1['12th'] = (pivot1["12th"]).map("{:.2f}".format)
pivot1 = pivot1[["9th", "10th", "11th", '12th']]

In [23]:
# group by grade and school name for average reading score table 
avg_reading_score = complete_school_data.groupby(['school_name', 'grade'])['reading_score'].mean().reset_index()
pivot2 = avg_reading_score.pivot_table(index='school_name', columns='grade', values='reading_score')
# formatting data to display
pivot2['9th'] = (pivot2["9th"]).map("{:.2f}".format)
pivot2['10th'] = (pivot2["10th"]).map("{:.2f}".format)
pivot2['11th'] = (pivot2["11th"]).map("{:.2f}".format)
pivot2['12th'] = (pivot2["12th"]).map("{:.2f}".format)
pivot2 = pivot2[["9th", "10th", "11th", '12th']]

## Average Math scores by grade

In [24]:
# Display average math scores by grade
pivot1

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Average reading scores by grade

In [25]:
# Display average reading by grade
pivot2

grade,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


##### We formatted the data so that it could be used in a summary of spending by each student

In [26]:
# binning the data
spending_bins = [0, 585, 630, 645, 680]
labels = ["$000-$585", "$585-630", "$630-645", "$645-680"]
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per student budget'], bins=spending_bins, labels=labels, include_lowest=True)

In [27]:
# setting the columns' names 
school_spending_df.rename(columns={'math_score' : 'Average math score', "reading_score" : "Average reading score"}, inplace=True)


In [28]:
# gathering the data
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average math score"].mean().reset_index()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average reading score"].mean().reset_index()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% passing math"].mean().reset_index()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% passing reading"].mean().reset_index()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean().reset_index()

In [29]:
# building the spending summary data frame
spending_data =[spending_reading_scores, spending_passing_math,
               spending_passing_reading, overall_passing_spending]

spending_summary= spending_math_scores

for frame in spending_data:
    spending_summary = spending_summary.merge(frame, how='right')

In [30]:
# formatting data to display
spending_summary['Average math score'] = (spending_summary["Average math score"]).map("{:.2f}".format)
spending_summary['Average reading score'] = (spending_summary["Average reading score"]).map("{:.2f}".format)
spending_summary['% passing math'] = (spending_summary["% passing math"]).map("{:.2f}%".format)
spending_summary['% passing reading'] = (spending_summary["% passing reading"]).map("{:.2f}%".format)
spending_summary['% Overall Passing'] = (spending_summary["% Overall Passing"]).map("{:.2f}%".format)

## Spending Summary
It’s important to note that the budget per student does not appear to be directly related to academic achievement. In fact, there seems to be an inverse correlation between them

In [31]:
# Display spending summary
spending_summary

,Spending Ranges (Per Student),Average math score,Average reading score,% passing math,% passing reading,% Overall Passing
0,$000-$585,83.46,83.93,0.93%,0.97%,0.90%
1,$585-630,81.90,83.16,0.87%,0.93%,0.81%
2,$630-645,78.52,81.62,0.73%,0.84%,0.63%
3,$645-680,77.00,81.03,0.66%,0.81%,0.54%


##### We formatted the data so that it could be used to create a report on the number of students at each school.

In [32]:
# binning the data by size
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
school_size_df["school size"] = pd.cut(school_size_df['student_name'], bins=size_bins, labels=labels, include_lowest=True)
school_size_df.rename(columns={'math_score' : 'Average math score', "reading_score" : "Average reading score"}, inplace=True)


In [33]:
# gathering the data for size summary dataframe
size_math_scores = school_size_df.groupby(["school size"])["Average math score"].mean().reset_index()
size_reading_scores = school_size_df.groupby(["school size"])["Average reading score"].mean().reset_index()
size_passing_math = school_size_df.groupby(["school size"])["% passing math"].mean().reset_index()
size_passing_reading = school_size_df.groupby(["school size"])["% passing reading"].mean().reset_index()
size_overall_passing = school_size_df.groupby(["school size"])["% Overall Passing"].mean().reset_index()

In [34]:
# building the size summary data frame 
size_data =[size_reading_scores, size_passing_math,
               size_passing_reading, size_overall_passing]

size_summary= size_math_scores

for frame in size_data:
    size_summary = size_summary.merge(frame, how='right')

In [35]:
# formatting data to display
size_summary['Average math score'] = (size_summary["Average math score"]).map("{:.2f}".format)
size_summary['Average reading score'] = (size_summary["Average reading score"]).map("{:.2f}".format)
size_summary['% passing math'] = (size_summary["% passing math"]*100).map("{:.2f}%".format)
size_summary['% passing reading'] = (size_summary["% passing reading"]*100).map("{:.2f}%".format)
size_summary['% Overall Passing'] = (size_summary["% Overall Passing"]*100).map("{:.2f}%".format)

## Size Summary
It’s important to note that schools with over 2000 students tend to have very low academic achievement.

In [36]:
# Display size summary 
size_summary

,school size,Average math score,Average reading score,% passing math,% passing reading,% Overall Passing
0,Small (<1000),83.82,83.93,93.55%,96.10%,89.88%
1,Medium (1000-2000),83.37,83.86,93.60%,96.79%,90.62%
2,Large (2000-5000),77.75,81.34,69.96%,82.77%,58.29%


##### We formatted the data so that it could be used to create a report on the different types of schools in the district

In [37]:
# setting the columns for school type data frame 
school_type_df.rename(columns={'type' : 'school type', 'math_score' : 'Average math score', "reading_score" : "Average reading score"}, inplace=True)

In [38]:
# gathering the data for type summary
type_math_scores = school_type_df.groupby(["school type"])["Average math score"].mean().reset_index()
type_reading_scores = school_type_df.groupby(["school type"])["Average reading score"].mean().reset_index()
type_passing_math = school_type_df.groupby(["school type"])["% passing math"].mean().reset_index()
type_passing_reading = school_type_df.groupby(["school type"])["% passing reading"].mean().reset_index()
type_overall_passing = school_type_df.groupby(["school type"])["% Overall Passing"].mean().reset_index()

In [39]:
# building type summary data frame
type_data =[type_reading_scores, type_passing_math,
               type_passing_reading, type_overall_passing]

type_summary= type_math_scores

for frame in type_data:
    type_summary = type_summary.merge(frame, how='right')



In [40]:
#formatting data to display
type_summary['Average math score'] = (type_summary["Average math score"]).map("{:.2f}".format)
type_summary['Average reading score'] = (type_summary["Average reading score"]).map("{:.2f}".format)
type_summary['% passing math'] = (type_summary["% passing math"]*100).map("{:.2f}%".format)
type_summary['% passing reading'] = (type_summary["% passing reading"]*100).map("{:.2f}%".format)
type_summary['% Overall Passing'] = (type_summary["% Overall Passing"]*100).map("{:.2f}%".format)

## Type Summary
It’s important to note the difference in academic achievement between district and charter schools.

In [41]:
# Display type summary 
type_summary

,school type,Average math score,Average reading score,% passing math,% passing reading,% Overall Passing
0,Charter,83.47,83.90,93.62%,96.59%,90.43%
1,District,76.96,80.97,66.55%,80.80%,53.67%


## Conclusions 
* Charter schools have been shown to have better academic performance than district schools in math and reading reading. They are able to achieve this with modest budget due to their better resource administration

* The district schools are able to have more studets that charter schools, but heving a greater budget per student and worst academic performance, being this perform signicantly inferior